In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import os

In [2]:
tf.__version__

'2.12.0'

In [3]:
! mkdir ~/.kaggle

! cp kaggle.json ~/.kaggle/

In [4]:
! chmod 600 ~/.kaggle/kaggle.json

In [5]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

# Import helper functions we're going to use
from helper_functions import create_tensorboard_callback, plot_loss_curves, unzip_data, walk_through_dir

--2023-07-03 11:01:54--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2023-07-03 11:01:54 (87.9 MB/s) - ‘helper_functions.py’ saved [10246/10246]



In [6]:
!kaggle datasets download -d iamsouravbanerjee/animal-image-dataset-90-different-animals

100% 656M/656M [00:29<00:00, 25.0MB/s]
100% 656M/656M [00:29<00:00, 23.0MB/s]


In [7]:
unzip_data("/content/animal-image-dataset-90-different-animals.zip")

In [8]:
dataset=tf.keras.preprocessing.image_dataset_from_directory('animals/animals',image_size=(224,224),label_mode='categorical')

Found 5400 files belonging to 90 classes.


In [9]:
class_names=dataset.class_names

In [10]:
class_names[:10]

['antelope',
 'badger',
 'bat',
 'bear',
 'bee',
 'beetle',
 'bison',
 'boar',
 'butterfly',
 'cat']

In [11]:
train_dataset=dataset.take(int(0.75*len(dataset)))
test_dataset=dataset.skip(int(0.75*len(dataset)))

In [12]:
len(train_dataset),len(test_dataset)

(126, 43)

In [12]:
train_dataset

<_TakeDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 90), dtype=tf.float32, name=None))>

In [13]:
train_data=train_dataset.shuffle(1000).prefetch(tf.data.AUTOTUNE)
test_data=test_dataset.shuffle(1000).prefetch(tf.data.AUTOTUNE)

In [ ]:
for image,label in train_data.take(1):
  print(label[0])

tf.Tensor(
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.], shape=(90,), dtype=float32)


In [14]:
data_augmentation_and_rescaling=keras.Sequential([
    layers.Resizing(224,224),
    layers.Rescaling(1./255)



],name="data_augmentation_layer")

In [15]:
n_classes=90

base_model=tf.keras.applications.ResNet50V2(include_top=False)
base_model.trainable=False
inputs=keras.Input(shape=(None,None,3),name='input_layer')
x=data_augmentation_and_rescaling(inputs)
x=base_model(x,training=False)
x=layers.GlobalAveragePooling2D(name='global_pooling_layer')(x)
outputs=layers.Dense(n_classes,name='output_layer',activation='softmax')(x)
model=keras.Model(inputs,outputs)


94668760/94668760 [==============================] - 5s 0us/step


In [16]:
model.compile(loss='categorical_crossentropy',metrics=['accuracy'],optimizer='adam')
history_animals=model.fit(train_data,epochs=3)

Epoch 1/3
126/126 [==============================] - 40s 91ms/step - loss: 1.7460 - accuracy: 0.6014
Epoch 2/3
126/126 [==============================] - 31s 95ms/step - loss: 0.2863 - accuracy: 0.9417
Epoch 3/3
126/126 [==============================] - 30s 94ms/step - loss: 0.1259 - accuracy: 0.9849


In [17]:
model.evaluate(test_data)

43/43 [==============================] - 29s 122ms/step - loss: 0.4122 - accuracy: 0.8904


[0.41217848658561707, 0.890350878238678]

In [11]:
def predict_image(filepath):
  image=tf.io.read_file(filepath)
  image=tf.io.decode_image(image)
  image=tf.expand_dims(image,axis=0)

  return class_names[model.predict(image).argmax()]

In [12]:

predict_image("/content/images.jpg")

1/1 [==============================] - 3s 3s/step


'bear'

In [18]:
model.save('animal_model')

In [ ]:
from tensorflow.keras.models import load_model
from tensorflow.lite import TFLiteConverter

ImportError: ignored

In [13]:
tf.saved_model.save(model,"")
converter=tf.lite.TFLiteConverter.from_saved_model("")
tflite_model=converter.convert()
with open('animal_model.tflite','wb') as f:
  f.write(tflite_model)

In [ ]:
class_names

['antelope',
 'badger',
 'bat',
 'bear',
 'bee',
 'beetle',
 'bison',
 'boar',
 'butterfly',
 'cat',
 'caterpillar',
 'chimpanzee',
 'cockroach',
 'cow',
 'coyote',
 'crab',
 'crow',
 'deer',
 'dog',
 'dolphin',
 'donkey',
 'dragonfly',
 'duck',
 'eagle',
 'elephant',
 'flamingo',
 'fly',
 'fox',
 'goat',
 'goldfish',
 'goose',
 'gorilla',
 'grasshopper',
 'hamster',
 'hare',
 'hedgehog',
 'hippopotamus',
 'hornbill',
 'horse',
 'hummingbird',
 'hyena',
 'jellyfish',
 'kangaroo',
 'koala',
 'ladybugs',
 'leopard',
 'lion',
 'lizard',
 'lobster',
 'mosquito',
 'moth',
 'mouse',
 'octopus',
 'okapi',
 'orangutan',
 'otter',
 'owl',
 'ox',
 'oyster',
 'panda',
 'parrot',
 'pelecaniformes',
 'penguin',
 'pig',
 'pigeon',
 'porcupine',
 'possum',
 'raccoon',
 'rat',
 'reindeer',
 'rhinoceros',
 'sandpiper',
 'seahorse',
 'seal',
 'shark',
 'sheep',
 'snake',
 'sparrow',
 'squid',
 'squirrel',
 'starfish',
 'swan',
 'tiger',
 'turkey',
 'turtle',
 'whale',
 'wolf',
 'wombat',
 'woodpecker',


In [19]:
!zip -r /content/file.zip /content/animal_model

  adding: content/animal_model/ (stored 0%)
  adding: content/animal_model/fingerprint.pb (stored 0%)
  adding: content/animal_model/assets/ (stored 0%)
  adding: content/animal_model/keras_metadata.pb (deflated 96%)
  adding: content/animal_model/variables/ (stored 0%)
  adding: content/animal_model/variables/variables.data-00000-of-00001 (deflated 7%)
  adding: content/animal_model/variables/variables.index (deflated 77%)
  adding: content/animal_model/saved_model.pb (deflated 92%)
